### Opening hours example

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import time
import pandas as pd

# --- 1️⃣ SPARQL: hämta Wikidata-objekt kopplade till SAT ---
query = """
SELECT ?item ?itemLabel ?coord ?osmRel ?osmWay ?osmNode WHERE {
  ?item wdt:P6104 wd:Q134294510;
        wdt:P625 ?coord.
  OPTIONAL { ?item wdt:P402 ?osmRel. }      # OSM relation
  OPTIONAL { ?item wdt:P11693 ?osmNode. }   # OSM node
  OPTIONAL { ?item wdt:P10689 ?osmWay. }    # OSM way
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],sv,en". }
}
"""

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

items = []
for r in results["results"]["bindings"]:
    qid = r["item"]["value"].split("/")[-1]
    label = r.get("itemLabel", {}).get("value", "")
    osm_node = r.get("osmNode", {}).get("value", "")
    osm_way = r.get("osmWay", {}).get("value", "")
    osm_rel = r.get("osmRel", {}).get("value", "")
    items.append({
        "qid": qid,
        "label": label,
        "osm_node": osm_node,
        "osm_way": osm_way,
        "osm_rel": osm_rel
    })

print(f"Hittade {len(items)} Wikidata-objekt kopplade till SAT.")


# --- 2️⃣ Funktion för att hämta OSM-taggar via Overpass ---
def get_osm_tags(osm_type, osm_id):
    query = f"""
    [out:json][timeout:25];
    {osm_type}({osm_id});
    out tags;
    """
    url = "https://overpass-api.de/api/interpreter"
    r = requests.get(url, params={"data": query})
    if r.status_code != 200:
        return {}
    data = r.json()
    if not data.get("elements"):
        return {}
    return data["elements"][0].get("tags", {})


# --- 3️⃣ Hämta opening_hours, description m.m. för varje OSM-objekt ---
records = []
for item in items:
    for osm_type, osm_id in [
        ("node", item["osm_node"]),
        ("way", item["osm_way"]),
        ("relation", item["osm_rel"])
    ]:
        if not osm_id:
            continue

        tags = get_osm_tags(osm_type, osm_id)
        if not tags:
            continue

        relevant = {k: v for k, v in tags.items()
                    if k in ["opening_hours", "opening_hours:note", "description", "source:opening_hours", "website"]}

        if relevant:
            record = {
                "qid": item["qid"],
                "label": item["label"],
                "osm_type": osm_type,
                "osm_id": osm_id,
                **relevant
            }
            records.append(record)
            print(f"✅ {item['label']} ({osm_type} {osm_id}) → {list(relevant.keys())}")
        time.sleep(1)  # vänlig paus mot Overpass

# --- 4️⃣ Sammanställ som tabell ---
df = pd.DataFrame(records)
print("\n--- Resultat ---")
print(df)

# --- 5️⃣ Spara till CSV (valfritt) ---
df.to_csv("SAT_opening_hours_OSM.csv", index=False, encoding="utf-8-sig")
print("\n💾 Sparat till SAT_opening_hours_OSM.csv")


Hittade 671 Wikidata-objekt kopplade till SAT.
✅ Grillplats, Storsand Ålö (node 1412263113) → ['website']
✅ Grillplats vid badstrand, Grinda (node 12759232983) → ['website']
✅ Grillplats Grinda tältplats syd västra 2 (node 11145662490) → ['website']
✅ Östra grillplatsen vid tältplatsen 2 (node 11145662491) → ['website']
✅ Rast- och grillplats Öja-Landsort (node 12662333661) → ['description', 'website']
✅ Stockholm Archipelago Trail (relation 19012437) → ['website']
✅ Arholma minneslund (way 513323289) → ['website']
✅ SAT Lidö (relation 19020231) → ['website']
✅ SAT Landsort (relation 19013576) → ['website']
✅ Grinda Wärdshus (node 4428110501) → ['website']
✅ SAT Nämdö (relation 19013473) → ['website']
✅ SAT Rånö (relation 19015969) → ['website']
✅ SAT Ornö (relation 19023687) → ['website']
✅ SAT Fjärdlång (relation 19016280) → ['website']
✅ SAT Svartsö (relation 19014515) → ['website']
✅ SAT Brottö (relation 19141225) → ['website']
✅ SAT Yxlan (relation 19020310) → ['website']
✅ SAT Mö

In [ ]:
items